In [1]:
import pandas as pd
import requests
import dlt
import io
from loguru import logger
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
def get_months_between_dates(start_date: str, end_date: str):
    start = datetime.strptime(start_date, '%Y-%m')
    end = datetime.strptime(end_date, '%Y-%m')
    
    months = []
    
    while start <= end:
        months.append(start.strftime('%Y-%m'))
        start += relativedelta(months=1)
    
    return months

months_green_yellow = get_months_between_dates('2019-01', '2020-12')
months_fhv = get_months_between_dates('2019-01', '2019-12')

In [3]:
green_taxi_url = 'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_{month}.csv.gz'
yellow_taxi_url = 'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/yellow/yellow_tripdata_{month}.csv.gz'
fhv_taxi_url = 'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhv/fhv_tripdata_{month}.csv.gz'

In [11]:
def load_data_from_url(url: str,
                       is_chunks: bool=False,
                       chunksize: int = 1000000):
    response = requests.get(url)
    response.raise_for_status()
    if is_chunks:
        for chunk in pd.read_csv(filepath_or_buffer=io.BytesIO(response.content), compression='gzip', chunksize=chunksize):
            yield chunk
    else:
        return pd.read_csv(filepath_or_buffer=io.BytesIO(response.content), compression='gzip')

In [12]:
pipeline = dlt.pipeline(
    pipeline_name='taxi_data_to_postgres',
    destination='postgres',
    dataset_name='public'
)

In [ ]:
# upload green_taxi data

for month in months_green_yellow:
    df_green = load_data_from_url(green_taxi_url.format(month=month))
    
    pipeline.run(
        data=df_green,
        table_name='green_taxi',
        write_disposition='append'
        )
    logger.info(f'inserted green data for {month}')

In [ ]:
# upload yellow_taxi data

for month in months_green_yellow:
    count_chunks = 0
    for chunk in load_data_from_url(
        url=yellow_taxi_url.format(month=month),
        is_chunks=True
        ):
        try:
            pipeline.run(
                data=chunk,
                table_name='yellow_taxi',
                write_disposition='append',
                )
            count_chunks += 1  
        except Exception as e:
            print(f'Error loading chunk : {count_chunks} for {month}')
    logger.info(f'inserted yellow data for {month}, chunks_count: {count_chunks}')

2025-02-23 12:30:40.286 | INFO     | __main__:<module>:13 - inserted yellow data for 2019-01, chunks_count: 8
2025-02-23 12:33:32.781 | INFO     | __main__:<module>:13 - inserted yellow data for 2019-02, chunks_count: 8
2025-02-23 12:36:52.022 | INFO     | __main__:<module>:13 - inserted yellow data for 2019-03, chunks_count: 8
2025-02-23 12:39:57.616 | INFO     | __main__:<module>:13 - inserted yellow data for 2019-04, chunks_count: 8
2025-02-23 12:43:37.142 | INFO     | __main__:<module>:13 - inserted yellow data for 2019-05, chunks_count: 8
2025-02-23 12:46:06.788 | INFO     | __main__:<module>:13 - inserted yellow data for 2019-06, chunks_count: 7
/tmp/ipykernel_4829/879967819.py:5: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(filepath_or_buffer=io.BytesIO(response.content), compression='gzip', chunksize=1000000):
2025-02-23 12:49:41.134 | INFO     | __main__:<module>:13 - inserted yellow data for 201

In [ ]:
# upload fhv dataset

for month in months_fhv:
    df_fhv = load_data_from_url(fhv_taxi_url.format(month=month))

    pipeline.run(
        data=df_fhv,
        table_name='fhv_taxi',
        write_disposition='append'
        )
    logger.info(f'inserted fhv data for {month}')

2025-02-23 01:11:00.231 | INFO     | __main__:<module>:9 - inserted fhv data for 2019-01
2025-02-23 01:11:29.282 | INFO     | __main__:<module>:9 - inserted fhv data for 2019-02
2025-02-23 01:11:49.786 | INFO     | __main__:<module>:9 - inserted fhv data for 2019-03
2025-02-23 01:12:09.709 | INFO     | __main__:<module>:9 - inserted fhv data for 2019-04
2025-02-23 01:12:34.463 | INFO     | __main__:<module>:9 - inserted fhv data for 2019-05
2025-02-23 01:13:15.690 | INFO     | __main__:<module>:9 - inserted fhv data for 2019-06
2025-02-23 01:14:05.124 | INFO     | __main__:<module>:9 - inserted fhv data for 2019-07
2025-02-23 01:14:53.553 | INFO     | __main__:<module>:9 - inserted fhv data for 2019-08
2025-02-23 01:15:25.655 | INFO     | __main__:<module>:9 - inserted fhv data for 2019-09
2025-02-23 01:15:50.796 | INFO     | __main__:<module>:9 - inserted fhv data for 2019-10
2025-02-23 01:16:10.126 | INFO     | __main__:<module>:9 - inserted fhv data for 2019-11
2025-02-23 01:16:30.4

In [ ]:
# upload taxi_zone_lookup
# better use dbt seed

df = pd.read_csv(
    filepath_or_buffer='../zoomcamp/seeds/taxi_zone_lookup.csv'
)

pipeline.run(
    data=df,
    table_name='taxi_zone_lookup',
    write_disposition='append'
    )